# Radial Average
## Importing libraries

In [1]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from tqdm import tqdm
from scipy.special import erf
import pickle
import itertools
from SALib.sample import saltelli
from SALib.analyze import sobol

# Personal libraries
import henon_map as hm

from parameters import *

## Computing

In [2]:
error_2 = {}
DA_2 = {}
error_5 = {}
DA_5 = {}
raw_error_2 = {}

alpha_preliminary_values = np.linspace(-1.0, 1.0, samples)
alpha_values = np.arccos(alpha_preliminary_values) / 2
d_preliminar_alpha = alpha_preliminary_values[1] - alpha_preliminary_values[0]

for epsilon in tqdm(epsilons, desc="Radial average"):
    # Extracting the radiuses with theta1 = theta2 = 0.0
    
    engine = hm.radial_scan.generate_instance(
        d_r, 
        alpha_values, 
        np.zeros(alpha_values.shape),
        np.zeros(alpha_values.shape),
        epsilon,
        starting_position=starting_position
    )
    all_radiuses = engine.compute(turn_sampling)
    
    # Computing of the 2D values
    
    skips = [1]
    while True:
        if (samples - 1) // skips[-1] > 4:
            skips.append(skips[-1] * 2)
        else:
            break
    
    for i in skips:
        alpha_prelim = alpha_preliminary_values[::i]
        cutted_radiuses = all_radiuses[::i]
        value = integrate.simps(cutted_radiuses ** 2, alpha_prelim, axis=0)
        less_value = integrate.simps(cutted_radiuses[::2] ** 2, alpha_prelim[::2], axis=0)
        uncertainty = np.abs((value - less_value))
        
        DA = np.sqrt(value / 2)
        uncertainty = 0.5 * np.power(value / 2, -0.5) * uncertainty
        DA_5[(epsilon, cutted_radiuses.shape)] = np.asarray(DA)
        error_5[(epsilon, cutted_radiuses.shape)] = uncertainty 
    
    # Rest of the Angular Averaging process
    
    values = []
    indexes = []
    raw_values = []
    refined_values = []
    for i in tqdm(range(len(turn_sampling)), desc="computing full track"):
        temp_values = np.array([[]])
        
        engine = hm.full_track.generate_instance(
            all_radiuses[:,i],
            alpha_values,
            np.zeros(alpha_values.shape),
            np.zeros(alpha_values.shape),
            np.ones(alpha_values.shape, dtype=np.int) * turn_sampling[i],
            epsilon)
        print("GPU")
        engine.new_compute()
        print("CPU")
        engine.accumulate_and_return(n_subdivisions)

        list_count_matrix = engine.count_matrix.copy()
        list_avg_matrix = engine.matrices.copy()
        
        _, _, result_total, validity_total = hm.cpu_accumulate_and_return(list_count_matrix, list_avg_matrix)
        
        values.append(result_total)
        indexes.append(np.argmax(validity_total, axis=0))
        refined_values.append([result_total[indexes[-1][i]][i] for i in range(len(indexes[-1]))])
        raw_values.append(np.average(np.power(engine.get_data()[0], 4), axis=0))
        # count_matrix_2[(epsilon, samples, turn_sampling[i])] = list_count_matrix
        # average_matrix_2[(epsilon, samples, turn_sampling[i])] = list_avg_matrix
    
    steps = [1]
    while True:
        if (values[0][0].shape[0] - 1) / steps[-1] > 4:
            steps.append(steps[-1] * 2)
        else:
            break

    for jump in steps:
        DA = []
        error = []
        DA_mc = []
        raw_error_mc = []
        error_mc = []
        for i in range(len(turn_sampling)):
            DA.append(np.power(integrate.simps(refined_values[i][::jump], alpha_preliminary_values[::jump]) * 0.5, 1/4))
            temp = np.power(integrate.simps(refined_values[i][::jump * 2], alpha_preliminary_values[::jump * 2]) * 0.5, 1/4)
            error.append(np.absolute(DA[-1] - temp))

            DA_mc.append(np.power(np.average(refined_values[i][::jump]), 1/4))
            raw_error_mc.append(np.std(refined_values[i][::jump]))
            error_mc.append(0.25 * np.power(DA_mc[-1], -3) * np.std(values[i][::jump]) / np.sqrt(np.size(values[i][::jump])))

        DA_2[(epsilon, len(refined_values[i][::jump]), "refined", "int")] = DA
        error_2[(epsilon, len(refined_values[i][::jump]), "refined", "int")] = error
        DA_2[(epsilon, len(refined_values[i][::jump]), "refined", "mc")] = DA_mc
        error_2[(epsilon, len(refined_values[i][::jump]), "refined", "mc")] = error_mc
        raw_error_2[(epsilon, len(refined_values[i][::jump]), "refined", "mc")] = raw_error_mc
        
        DA = []
        error = []
        DA_mc = []
        raw_error_mc = []
        error_mc = []
        for i in range(len(turn_sampling)):
            DA.append(np.power(integrate.simps(raw_values[i][::jump], alpha_preliminary_values[::jump]) * 0.5, 1/4))
            temp = np.power(integrate.simps(raw_values[i][::jump * 2], alpha_preliminary_values[::jump * 2]) * 0.5, 1/4)
            error.append(np.absolute(DA[-1] - temp))

            DA_mc.append(np.power(np.average(raw_values[i][::jump]), 1/4))
            raw_error_mc.append(np.std(raw_values[i][::jump]))
            error_mc.append(0.25 * np.power(DA_mc[-1], -3) * np.std(values[i][::jump]) / np.sqrt(np.size(values[i][::jump])))

        DA_2[(epsilon, len(raw_values[i][::jump]), "raw", "int")] = DA
        error_2[(epsilon, len(raw_values[i][::jump]), "raw", "int")] = error
        DA_2[(epsilon, len(raw_values[i][::jump]), "raw", "mc")] = DA_mc
        error_2[(epsilon, len(raw_values[i][::jump]), "raw", "mc")] = error_mc
        raw_error_2[(epsilon, len(raw_values[i][::jump]), "raw", "mc")] = raw_error_mc
        
        for j in range(len(values[0])):
            DA = []
            error = []
            DA_mc = []
            raw_error_mc = []
            error_mc = []
            for i in range(len(turn_sampling)):
                DA.append(np.power(integrate.simps(values[i][j][::jump], alpha_preliminary_values[::jump]) * 0.5, 1/4))
                temp = np.power(integrate.simps(values[i][j][::jump * 2], alpha_preliminary_values[::jump * 2]) * 0.5, 1/4)
                error.append(np.absolute(DA[-1] - temp))
                
                DA_mc.append(np.power(np.average(values[i][j][::jump]), 1/4))
                raw_error_mc.append(np.std(values[i][j][::jump]))
                error_mc.append(0.25 * np.power(DA_mc[-1], -3) * np.std(values[i][j][::jump]) / np.sqrt(np.size(values[i][j][::jump])))
                
            DA_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "int")] = DA
            error_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "int")] = error
            DA_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "mc")] = DA_mc
            error_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "mc")] = error_mc
            raw_error_2[(epsilon, len(values[i][j][::jump]), 2 ** (j), "mc")] = raw_error_mc

computing full track:   0%|          | 0/101 [00:00<?, ?it/s]

GPU
CPU


/home/carlidel/Insync/carlo.montanari3@studio.unibo.it/OneDrive Biz/optimized_code/henon_map/henon_map/cpu_henon_core.py:153: RuntimeWarning: invalid value encountered in true_divide
  matrices = np.nansum(matrices.reshape(

computing full track:   1%|          | 1/101 [00:48<1:21:18, 48.78s/it]

GPU
CPU



computing full track:   2%|▏         | 2/101 [01:33<1:18:19, 47.47s/it]

...


computing full track:  98%|█████████▊| 99/101 [41:53<00:11,  5.63s/it]

GPU
CPU



computing full track:  99%|█████████▉| 100/101 [41:57<00:05,  5.17s/it]

GPU
CPU



Radial average: 100%|██████████| 1/1 [42:57<00:00, 2577.29s/it]


## Saving data

In [3]:
with open(savepath + "data/DA_2.pkl", 'wb') as f:
    pickle.dump(DA_2, f, protocol=4)
    
with open(savepath + "data/error_2.pkl", 'wb') as f:
    pickle.dump(error_2, f, protocol=4)

with open(savepath + "data/raw_error_2.pkl", 'wb') as f:
    pickle.dump(raw_error_2, f, protocol=4)    

with open(savepath + "data/DA_5.pkl", 'wb') as f:
    pickle.dump(DA_5, f, protocol=4)
    
with open(savepath + "data/error_5.pkl", 'wb') as f:
    pickle.dump(error_5, f, protocol=4)